In [71]:
import math
from functools import reduce
import operator as op
from fractions import Fraction

In [76]:
# non-terminal 정의
digit = [0,1,2,3,4,5,6,7,8,9]
assignment_op = ':='
semi_colon = ';'
add_operator = ['+', '-']
mult_operator = ['*', '/']
left_paren = '('
right_paren = ')'

# terminal 정의
ID = ['ident']
OP = ['add_operator', 'mult_operator']
CONST = ['digit']

In [79]:
# LEXER 함수
def tokenize(string):
    dic = {}
    tokens = string.split(' ')
    for token in tokens:
        if token[0] in str(digit):
            dic[token] = "digit"
        elif token in assignment_op:
            dic[token] = "assignment_op"
        elif token in semi_colon:
            dic[token] = "semi_colon"
        elif token in add_operator:
            if token[0] == '-' and len(token) > 1:
                dic[token] = "digit"
            else:
                dic[token] = "add_operator"
        elif token in mult_operator:
            dic[token] = "mult_operator"
        elif token in left_paren:
            dic[token] = "left_paren"
        elif token in right_paren:
            dic[token] = "right_paren"
        else:
            dic[token] = "ident"
    return dic

In [93]:
import copy
import sys

RULE = [
    ['program', 'statements'],
    ['statements', 'statement'],
    ['statements', 'statement semi_colon statements'],
    ['statement', 'ID assignment_op expression'],
    ['expression', 'term term_tail'],
    ['term_tail', 'add_op term term_tail'],
    ['term_tail', 'epsilon'],
    ['term', 'factor factor_tail'],
    ['factor_tail', 'mult_op factor factor_tail'],
    ['factor_tail', 'epsilon'],
    ['factor', 'left_paren expression right_paren'],
    ['factor', 'ID'],
    ['factor', 'CONST'],
    ['CONST', 'const'],
    ['ID', 'id'],
    ['assignment', ':='],
    ['semi_colon', ';'],
    ['add_op', '+'],
    ['add_op', '-'],
    ['mult_op', '*'],
    ['mult_op', '/'],
    ['left_paren', '('],
    ['right_paren', ')']
]

In [94]:
# SLR table 정의
SLR_TABLE = [
    {'id': 'S4', 'statements': 1, 'statement': 2, 'ID': 3, },
    {'$': 'ACC', },
    {';': 'S6', '$': 'R1', 'semi_colon': 5, },
    {':=': 'S8', 'assignment_op': 7, },
    {':=': 'R14', ';': 'R14', '+': 'R14', '-': 'R14', '*': 'R14', '/': 'R14', ')': 'R14', '$': 'R14', },
    {'id': 'S4', 'statements': 9, 'statement': 2, 'ID': 3, },
    {'id': 'R16', },
    {'const': 'S17', 'id': 'S4', '(': 'S16', 'expression': 10, 'term': 11, 'factor': 12, 'CONST': 15, 'ID': 14, 'left_paren': 13, },
    {'const': 'R15', 'id': 'R15', '(': 'R15', },
    {'$': 'R2', },
    {';': 'R3', '$': 'R3', },
    {';': 'R6', '+': 'S20', '-': 'S21', ')': 'R6', '$': 'R6', 'term_tail': 18, 'add_op': 19, },
    {';': 'R9', '+': 'R9', '-': 'R9', '*': 'S24', '/': 'S25', ')': 'R9', '$': 'R9', 'factor_tail': 22, 'mult_op': 23, },
    {'const': 'S17', 'id': 'S4', '(': 'S16', 'expression': 26, 'term': 11, 'factor': 12, 'CONST': 15, 'ID': 14, 'left_paren': 13, },
    {';': 'R11', '+': 'R11', '-': 'R11', '*': 'R11', '/': 'R11', ')': 'R11', '$': 'R11', },
    {';': 'R12', '+': 'R12', '-': 'R12', '*': 'R12', '/': 'R12', ')': 'R12', '$': 'R12', },
    {'const': 'R21', 'id': 'R21', '(': 'R21', },
    {';': 'R13', '+': 'R13', '-': 'R13', '*': 'R13', '/': 'R13', ')': 'R13', '$': 'R13', },
    {';': 'R4', ')': 'R4', '$': 'R4', },
    {'const': 'S17', 'id': 'S4', '(': 'S16', 'term': 27, 'factor': 12, 'CONST': 15, 'ID': 14, 'left_paren': 13, },
    {'const': 'R17', 'id': 'R17', '(': 'R17', },
    {'const': 'R18', 'id': 'R18', '(': 'R18', },
    {';': 'R7', '+': 'R7', '-': 'R7', ')': 'R7', '$': 'R7', },
    {'const': 'S17', 'id': 'S4', '(': 'S16', 'factor': 28, 'CONST': 15, 'ID': 14, 'left_paren': 13, },
    {'const': 'R19', 'id': 'R19', '(': 'R19', },
    {'const': 'R20', 'id': 'R20', '(': 'R20', },
    {')': 'S30', 'right_paren': 29, },
    {';': 'R6', '+': 'S20', '-': 'S21', ')': 'R6', '$': 'R6', 'term_tail': 31, 'add_op': 19, },
    {';': 'R9', '+': 'R9', '-': 'R9', '*': 'S24', '/': 'S25', ')': 'R9', '$': 'R9', 'factor_tail': 32, 'mult_op': 23, },
    {';': 'R10', '+': 'R10', '-': 'R10', '*': 'R10', '/': 'R10', ')': 'R10', '$': 'R10', },
    {';': 'R22', '+': 'R22', '-': 'R22', '*': 'R22', '/': 'R22', ')': 'R22', '$': 'R22', },
    {';': 'R5', ')': 'R5', '$': 'R5', },
    {';': 'R8', '+': 'R8', '-': 'R8', ')': 'R8', '$': 'R8', },
]

In [97]:
def parser():
    if len(terminal_list) == 1:
        return True, ''

    SLR_stack = [0]
    splitter = 0
    error_line = 1
    check = 0

    while True:
        next_input_symbol = terminal_list[splitter]
        check += 1
        current_state = SLR_stack[-1]

        if current_state == 1 and next_input_symbol == '$':
            print("ACCEPTED!!!")
            return True

        # 판단할 수 없는 symbol이 있으면 error
        if next_input_symbol not in SLR_TABLE[current_state].keys():
            error = "[REJECTED] Error in line " + str(error_line) + ", " + error_checker[error_line-1]
            print(error)
            return False, error

        # shift
        if SLR_TABLE[current_state][next_input_symbol][0] == 'S':
            splitter += 1
            error_line += 1
            SLR_stack.append(int(SLR_TABLE[current_state][next_input_symbol][1:]))

        # Reduce
        elif SLR_TABLE[current_state][next_input_symbol][0] == 'R':
            table_num = SLR_TABLE[current_state][next_input_symbol][1:]

            non_termi = RULE[int(table_num)][0]  # ex) CODE → epsilon 에서 VDECL 의미
            termi = RULE[int(table_num)][1]  # ex) CODE → epsilon 에서 epsilon 의미
            termi_length = len(str(termi).split(" "))  # 룰의 어절의 개수  ex) VDECL → vtype id semi : 3

            for i in range(termi_length):
                if termi != 'epsilon':
                    SLR_stack.pop()
                    terminal_list.pop(splitter-i-1)

            if non_termi not in SLR_TABLE[SLR_stack[-1]].keys():
                error = "[REJECTED] Error in line " + str(error_line) + ", " + error_checker[error_line - 1]
                print(error)
                return False, error

            # GOTO
            SLR_stack.append(SLR_TABLE[SLR_stack[-1]][non_termi])

            if termi != 'epsilon':
                splitter = splitter - termi_length + 1
            else:
                splitter += 1

            terminal_list.insert(splitter-1, non_termi)
